### Exercise 1:

- Create a new repository named regression-exercises in your GitHub; all of your Regression work will be housed here.
- Clone this repository within your local codeup-data-science directory.
- Create a .gitignore and make sure your list of 'files to ignore' includes your env.py file.
- Ceate a README.md file that outlines the contents and purpose of your repository.
- Add, commit, and push these two files.
- Now you can add your env.py file to this repository to access the Codeup database server.
- For these exercises, you will create wrangle.ipynb and wrangle.py files to hold necessary functions.


-----------------------

### Exercise 2

### Scenario:

#### As a customer analyst for Telco, you want to know who has spent the most money with the company over their lifetime. 
#### You have monthly charges and tenure, so you think you will be able to use those two attributes as features to estimate total charges. 
#### You need to do this within an average of $5.00 per customer.




1.) Acquire customer_id, monthly_charges, tenure, and total_charges from the telco_churn database for all customers with a 2-year contract.

2.) Using your acquired Telco data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above

3.)  You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

------------------------------

## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import acquire
import wrangle

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split

In [5]:
telco_df = acquire.get_telco_churn()

In [6]:
# shape of the dataframe is 14 records x 4 fields
telco_df.shape

(1695, 4)

In [7]:
# looking at actual df.
telco_df.head()

,customer_id,tenure,monthly_charges,total_charges
0,0013-SMEOE,71,109.70,7904.25
1,0014-BMAQU,63,84.65,5377.8
2,0016-QLJIS,65,90.45,5957.9
3,0017-DINOC,54,45.20,2460.55
4,0017-IUDMW,72,116.80,8456.75


In [38]:
#total charges is an object and needs to be changed to a float, other fields are numerical.
telco_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   tenure           1695 non-null   int64  
 2   monthly_charges  1695 non-null   float64
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


In [39]:
# can see the range of monthly_charges is ~100, need to change total to see the range.
telco_df.describe().T

,count,mean,std,min,25%,50%,75%,max
tenure,1695.0,56.735103,18.209363,0.0,48.000,64.00,71.00,72.00
monthly_charges,1695.0,60.770413,34.678865,18.4,24.025,64.35,90.45,118.75


#### Main Takeaways: 
- need to change total_charges to float.
- need to fix number of decimals places on monthly and total
- changes tenure to tenure months
- check for empty and null values.
    


 ---------------------------

## Prepare

In [40]:
#drop duplciates
telco_df.head()


,customer_id,tenure,monthly_charges,total_charges
0,0013-SMEOE,71,109.70,7904.25
1,0014-BMAQU,63,84.65,5377.8
2,0016-QLJIS,65,90.45,5957.9
3,0017-DINOC,54,45.20,2460.55
4,0017-IUDMW,72,116.80,8456.75


In [41]:
# shape before using drop
telco_df.shape

(1695, 4)

In [42]:
telco_df.drop_duplicates(inplace =True)

In [43]:
telco_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   tenure           1695 non-null   int64  
 2   monthly_charges  1695 non-null   float64
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 66.2+ KB


In [44]:
#check for nulls.
telco_df.isnull().sum()

customer_id        0
tenure             0
monthly_charges    0
total_charges      0
dtype: int64

In [45]:
# check for nans
telco_df.isna().sum()

customer_id        0
tenure             0
monthly_charges    0
total_charges      0
dtype: int64

In [46]:
# creatign df which has empty values both objecta and nunmerics replaced with a nan value.
telco_df= telco_df.replace(r'^\s*$', np.nan, regex=True)

In [47]:
# appears there is only 10 nan within total_charges, due to the size of the df, it seems safe to drop the value.
telco_df.isna().sum()

customer_id         0
tenure              0
monthly_charges     0
total_charges      10
dtype: int64

In [48]:
#looking at the rows which contain Nan values
Nan_records = telco_df[telco_df['total_charges'].isna()]
Nan_records

,customer_id,tenure,monthly_charges,total_charges
234,1371-DWPAZ,0,56.05,NaN
416,2520-SGTTA,0,20.00,NaN
453,2775-SEFEE,0,61.90,NaN
505,3115-CZMZD,0,20.25,NaN
524,3213-VVOLG,0,25.35,NaN
678,4075-WKNIU,0,73.35,NaN
716,4367-NUYAO,0,25.75,NaN
726,4472-LVYGI,0,52.55,NaN
941,5709-LVOEQ,0,80.85,NaN
1293,7644-OMVMY,0,19.85,NaN


In [49]:
#since all the NaN values occure in tenure of 0 months lets get the total count of the customers with tenure value of 0.
telco_df.groupby(['tenure']).count().T

tenure,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,72
customer_id,10,2,1,3,4,1,8,4,13,3,...,39,40,42,43,56,65,66,88,137,343
monthly_charges,10,2,1,3,4,1,8,4,13,3,...,39,40,42,43,56,65,66,88,137,343
total_charges,0,2,1,3,4,1,8,4,13,3,...,39,40,42,43,56,65,66,88,137,343


In [50]:
# replacing the NaN value in total_charges of the customers with 0 tenure with their monthly_charge.
telco_df.total_charges = telco_df.total_charges.fillna(telco_df.monthly_charges)

In [51]:
# total_charges needs to become a float value, so changing it using astype.
telco_df = telco_df.astype({'total_charges': 'float64'})

In [52]:
telco_df.groupby(['tenure']).count().T

tenure,0,1,2,3,4,5,6,7,8,9,...,63,64,65,66,67,68,69,70,71,72
customer_id,10,2,1,3,4,1,8,4,13,3,...,39,40,42,43,56,65,66,88,137,343
monthly_charges,10,2,1,3,4,1,8,4,13,3,...,39,40,42,43,56,65,66,88,137,343
total_charges,10,2,1,3,4,1,8,4,13,3,...,39,40,42,43,56,65,66,88,137,343


In [53]:
#can see change was successful
telco_df[telco_df['tenure'] == 0]

,customer_id,tenure,monthly_charges,total_charges
234,1371-DWPAZ,0,56.05,56.05
416,2520-SGTTA,0,20.00,20.00
453,2775-SEFEE,0,61.90,61.90
505,3115-CZMZD,0,20.25,20.25
524,3213-VVOLG,0,25.35,25.35
678,4075-WKNIU,0,73.35,73.35
716,4367-NUYAO,0,25.75,25.75
726,4472-LVYGI,0,52.55,52.55
941,5709-LVOEQ,0,80.85,80.85
1293,7644-OMVMY,0,19.85,19.85


In [54]:
# number of records is 1695 records x 4 fields.
telco_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   tenure           1695 non-null   int64  
 2   monthly_charges  1695 non-null   float64
 3   total_charges    1695 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 66.2+ KB


In [55]:
# train validate and test split function which doesnt utilize stratify.
def telco_split(df):
    '''
    This function take in the telco_churn_data acquired by get_telco_churn,
    performs a split and stratifies total_charges column.
    Returns train, validate, and test dfs.
    '''
    #20% test, 80% train_validate
    train_validate, test = train_test_split(df, test_size=0.2, 
                                        random_state=1349)
    # 80% train_validate: 30% validate, 70% train.
    train, validate = train_test_split(train_validate, train_size=0.7, 
                                   random_state=1349)
    return train, validate, test

#### - End with a wrangle.py file that contains the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values. Name your final function wrangle_telco.

In [2]:
telco_df = wrangle.wrangle_telco()

In [3]:
telco_df

,customer_id,tenure_months,monthly_charges,total_charges,tenure_year
0,0013-SMEOE,71,109.70,7904.25,6.0
1,0014-BMAQU,63,84.65,5377.80,5.0
2,0016-QLJIS,65,90.45,5957.90,5.0
3,0017-DINOC,54,45.20,2460.55,4.0
4,0017-IUDMW,72,116.80,8456.75,6.0
...,...,...,...,...,...
1690,9964-WBQDJ,71,24.40,1725.40,6.0
1691,9972-EWRJS,67,19.25,1372.90,6.0
1692,9975-GPKZU,46,19.75,856.50,4.0
1693,9993-LHIEB,67,67.85,4627.65,6.0


Exercises III - Challenge

Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.

As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.

In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.
Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaninful; remember to document your process and decisions using markdown and code commenting where helpful.
Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe witn no missing values in your wrangle.py file. Name your final function wrangle_zillow.